<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

## Packages

In [ ]:
!pip install --upgrade git+https://github.com/xiaoyufan/nbme.git

In [ ]:
from nbme_utils import scoring